In [88]:
from fscve_step1_db import AnalysisState

from fscve_step1_db import FscveStep1DbHelper
dbhelper = FscveStep1DbHelper( "nm", "fca",  1, False)
session = dbhelper.session
expid = dbhelper.experiment_id
fc = "fca"
bm = "nm"
benchmark = dbhelper.benchmark
print("***"*10,f" fscve_step0_pre_data bm:{bm}----fc:{fc}---expid:{expid}start")

******************************  fscve_step0_pre_data bm:nm----fc:fca---expid:1start


In [89]:
from sqlalchemy import desc
from casefc.fscve_step1_db import Discovery
query = session.query(Discovery).order_by(desc(Discovery.discovery_time)).first()
print(query)

Discovery(discovery_id=55167, test_case_hash=668cb1f849a506e42cbc07c2f97caaf6ae779df25022ba1c6c8c7bc029a5c57a, discovery_fuzzer=2560957982,  discovery_time=86390, is_new=1)


In [90]:
import numpy as np
from sqlalchemy import func
from casefc.fscve_step1_db import Discovery, Dispatch, EdgeCoverageFuzzer, EdgeCoverageGlobal, FuzzerEvent

query = session.query(Discovery.discovery_id).filter(Discovery.discovery_time > 86400).all()
# print(query[0].discovery_id)
#query all discoveries with time after 24h
arr_discovery = [res.discovery_id for res in query]
print(arr_discovery)

#delete the records after 24h , from table analysis_states,edge_coverages_fuzzer,edge_coverages_global,fuzzer_events,dispatch


print(session.query(Dispatch.test_case_hash).filter(Dispatch.dispatch_time > 86400).all())
session.query(Dispatch).filter(Dispatch.dispatch_time > 86400).delete(synchronize_session=False)
session.commit()
print("*******dispatch*******", session.query(Dispatch.test_case_hash).filter(Dispatch.dispatch_time > 86400).all())

print("*******AnalysisState*******"*6)
print(session.query(AnalysisState.discovery_id).filter(AnalysisState.discovery_id.in_(arr_discovery)).all())
session.query(AnalysisState).filter(AnalysisState.discovery_id.in_(arr_discovery)).delete(synchronize_session=False)
session.commit()
print("*******AnalysisState*******", session.query(AnalysisState.discovery_id).filter(AnalysisState.discovery_id.in_(arr_discovery)).all())

print("*****EdgeCoverageFuzzer*********"*6)
print(session.query(EdgeCoverageFuzzer).filter(EdgeCoverageFuzzer.discovery_id.in_(arr_discovery)).all())
session.query(EdgeCoverageFuzzer).filter(EdgeCoverageFuzzer.discovery_id.in_(arr_discovery)).delete(synchronize_session=False)
session.commit()
print("*******EdgeCoverageFuzzer*******", session.query(EdgeCoverageFuzzer.discovery_id).filter(EdgeCoverageFuzzer.discovery_id.in_(arr_discovery)).all())

print("*******EdgeCoverageGlobal*******"*6)
print(session.query(EdgeCoverageGlobal).filter(EdgeCoverageGlobal.discovery_id.in_(arr_discovery)).all())
session.query(EdgeCoverageGlobal).filter(EdgeCoverageGlobal.discovery_id.in_(arr_discovery)).delete(synchronize_session=False)
session.commit()
print("******EdgeCoverageGlobal********", session.query(EdgeCoverageGlobal.discovery_id).filter(EdgeCoverageGlobal.discovery_id.in_(arr_discovery)).all())

print("FuzzerEvent"*6)
print(session.query(FuzzerEvent).filter(FuzzerEvent.event_time > 86400).all())
session.query(FuzzerEvent).filter(FuzzerEvent.event_time > 86400).delete(synchronize_session=False)
session.commit()
print("*****FuzzerEvent*********", session.query(FuzzerEvent).filter(FuzzerEvent.event_time > 86400).all())


[]
[]
*******dispatch******* []
*******AnalysisState**************AnalysisState**************AnalysisState**************AnalysisState**************AnalysisState**************AnalysisState*******
[]
*******AnalysisState******* []
*****EdgeCoverageFuzzer**************EdgeCoverageFuzzer**************EdgeCoverageFuzzer**************EdgeCoverageFuzzer**************EdgeCoverageFuzzer**************EdgeCoverageFuzzer*********
[]
*******EdgeCoverageFuzzer******* []
*******EdgeCoverageGlobal**************EdgeCoverageGlobal**************EdgeCoverageGlobal**************EdgeCoverageGlobal**************EdgeCoverageGlobal**************EdgeCoverageGlobal*******
[]
******EdgeCoverageGlobal******** []
FuzzerEventFuzzerEventFuzzerEventFuzzerEventFuzzerEventFuzzerEvent
[]
*****FuzzerEvent********* []


In [91]:
import os
from casefc.pyutilszxy import walk_files
from casefcenfuzzcmp.fscve_step1_db import TestCase

print("******delete crash db cords and files after 24h ********start"*3)
query = session.query(Discovery.test_case_hash).filter(Discovery.discovery_time > 86400).all()
#query all discoveries with time after 24h
arr_test_case_hash = [res.test_case_hash for res in query]
print(arr_test_case_hash)
query = session.query(TestCase).filter(TestCase.hash.in_(arr_test_case_hash),TestCase.test_case_type_id == 2).all()
array_crashes = [res.hash for res in query]

#delete crash files after 24h
crash_dir = f"runinfosqlites/{benchmark}/{fc}/{expid}/crashes"
print(len(walk_files(crash_dir)))
for crash in array_crashes:
    path = f"{crash_dir}/{crash}"
    if os.path.exists(path):  # 如果文件存在
        # 删除文件，可使用以下两种方法。
        os.remove(path)
        #os.unlink(path)
    else:
        print('no such file:%s'%crash)  # 则返回文件不存在
print("after delete",len(walk_files(crash_dir)))

#delete all testcases after 24h
print("del testcase start",session.query(TestCase).filter(TestCase.hash.in_(arr_test_case_hash)).all())
session.query(TestCase).filter(TestCase.hash.in_(arr_test_case_hash)).delete(synchronize_session=False)
session.commit()
print("del testcase end",session.query(TestCase).filter(TestCase.hash.in_(arr_test_case_hash)).all())

print("******delete crash db records and files after 24h ********end"*3)

print("******delete discoveries after 24h ********start"*3,len(session.query(Discovery.test_case_hash).filter(Discovery.discovery_time > 86400).all()))
session.query(Discovery).filter(Discovery.discovery_time > 86400).delete(synchronize_session=False)
session.commit()
print("******delete discoveries after 24h ********end"*3,len(session.query(Discovery.test_case_hash).filter(Discovery.discovery_time > 86400).all()))

******delete crash db cords and files after 24h ********start******delete crash db cords and files after 24h ********start******delete crash db cords and files after 24h ********start
[]
2683
after delete 2683
del testcase start []
del testcase end []
******delete crash db records and files after 24h ********end******delete crash db records and files after 24h ********end******delete crash db records and files after 24h ********end
******delete discoveries after 24h ********start******delete discoveries after 24h ********start******delete discoveries after 24h ********start 0
******delete discoveries after 24h ********end******delete discoveries after 24h ********end******delete discoveries after 24h ********end 0


In [92]:
print("***"*10,f" fscve_step0_pre_data bm:{bm}----fc:{fc}---expid:{expid}end")

******************************  fscve_step0_pre_data bm:nm----fc:fca---expid:1end
